In [1]:
import sys
sys.path.append('..')
import doctable

In [35]:
import dataclasses
import datetime
import typing
@dataclasses.dataclass(repr=False)
class Row(doctable.DocTableSchema):
    payload: typing.Any
    idx: int = doctable.IDCol()
    updated: datetime.datetime = doctable.UpdatedCol()

class RowList(list):
    def show_rows(self):
        for el in self:
            print(f'{el}')
            
class RowGroup:
    def __init__(self, rows):
        self.dat = dict()
        for row in rows:
            k = row['payload'] % 10
            self.dat.setdefault(k, list())
            self.dat[k].append(row)
    def count(self):
        print({k:len(v) for k,v in self.dat.items()})
        

db = doctable.DocTable(schema=Row, target=':memory:')

print('insertion')
n = 1000
%timeit db.insert(list(Row(i) for i in range(n)))
%timeit db.insert(RowList(Row(i) for i in range(n)))
print(db.count())

print('select')
%timeit db.select()
%timeit db.select(result_container=RowList)
%timeit db.select(result_container=RowGroup)
''

insertion
26.5 ms ± 2.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
24.6 ms ± 1.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
162000
select
631 ms ± 6.05 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
628 ms ± 9.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
804 ms ± 36.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


''

In [25]:
isinstance(RowList, list)

False

In [38]:
class MySequence(list):
    def sum(self):
        return sum(self)

n = 1000000
    
print('base')
%timeit seq = list(range(n)) # x
%timeit seq = MySequence(range(n)) # x

print('comprehension')
%timeit seq = [i for i in range(n)] # x*1.3

print('from iterator')
%timeit seq = list(i for i in range(n)) # x*1.9
%timeit seq = MySequence(i for i in range(n)) # x*1.9

print('shallow copy')
%timeit seq = list([i for i in range(n)]) # x*2.0
%timeit seq = MySequence([i for i in range(n)]) # x*2.0
%timeit seq = MySequence(list(range(n))) # x*1.3

base
40.7 ms ± 3.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
39.4 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
comprehension
59.5 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
from iterator
73.8 ms ± 423 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
74.5 ms ± 1.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
shallow copy
70.5 ms ± 602 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
71.9 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
50.1 ms ± 750 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
